<a href="https://colab.research.google.com/github/you0229/kaggle_titanic/blob/master/main_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# libraries
import pandas as pd

In [0]:
# データ読み込み
def read_data():
  print('Train data reading...')
  train = pd.read_csv('/content/drive/My Drive/MachineLeaning/kaggle_titanic/train.csv')
  print("Train data is column {}, rows: {}".format(train.shape[0], train.shape[1]))
  print('Test data reading...')
  test = pd.read_csv('/content/drive/My Drive/MachineLeaning/kaggle_titanic/test.csv')
  print("Test data is column {}, rows: {}".format(test.shape[0], test.shape[1]))

  return train, test

In [0]:
# データ整理
def create_data(train, test):
  # train_test_combine = pd.concat(train.drop('Survived', axis=1),test)
  train['Ticket_Left'] = train['Ticket'].apply(lambda x: str(x)[0])
  display(train.head(3))
  return train

In [17]:
train = create_data(train, test)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_Left
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,A
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,P
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,S


In [18]:
display(train['Ticket_Left'].unique())

array(['A', 'P', 'S', '1', '3', '2', 'C', '7', 'W', '4', 'F', 'L', '9',
       '6', '5', '8'], dtype=object)

In [19]:
ticket_to_num = {'A':0, 'P':1, 'S':2, '1':3, '3':4, '2':5, 'C':6, '7':7, 'W':8, '4':9, 'F':9, 'L':10, '9':11,'6':12, '5':13, '8':14}
train['Ticket_Left'] = train['Ticket_Left'].translate(str.maketrans(ticket_to_num))

AttributeError: ignored

In [0]:
# モデル作成

In [0]:
# 学習

In [0]:
# 評価

In [10]:
train, test = read_data()
display(train.head(3))
display(test.head(3))

Train data reading...
Train data is column 891, rows: 12
Test data reading...
Test data is column 418, rows: 11


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
